This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'
import glob
from astropy.table import Table, vstack

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20190207'
file_list = glob.glob('../../dmu32/dmu32_XMM-LSS/data/tiles/xmm-lss_{}_cigale_*.fits'.format(SUFFIX))

In [3]:
#len(master_catalogue)

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
def main_help_sample(master_catalogue):
    good = {}
    for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
        good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
                                       ~master_catalogue['flag_{}'.format(band)])
        good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                                  master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)
    combined_good = np.sum(list(good.values()), axis=0) >= 2
    print("Number of good sources: {}".format(np.sum(combined_good)))
    return master_catalogue[combined_good].copy()

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
# combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
# print("Number of good sources: {}".format(np.sum(combined_good)))

# Main catalogue for CIGALE

In [7]:
# best_catalogue = master_catalogue[combined_good].copy()

In [8]:
# Correction for galactic extinction
#best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [9]:
# Convertion to CIGALE format
#best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [10]:
u_bands = [                         "megacam_u"]
g_bands = [            "suprime_g", "megacam_g", "decam_g", "gpc1_g" ]
r_bands = [            "suprime_r", "megacam_r", "decam_r", "gpc1_r" ]
i_bands = [            "suprime_i", "megacam_i", "decam_i", "gpc1_i" ]
z_bands = ["vista_z",  "suprime_z", "megacam_z", "decam_z", "gpc1_z" ]
y_bands = ["vista_y",  "suprime_y", "megacam_y", "decam_y", "gpc1_y" ]
j_bands = ["vista_j",  "wircam_j",  "ukidss_j"]
h_bands = ["vista_h",  "wircam_h",  "ukidss_h"]
k_bands = ["vista_ks", "wircam_ks"] #, "ukidss_k"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [11]:
# remove_unneeded_fluxes(g_bands)
# remove_unneeded_fluxes(u_bands)
# remove_unneeded_fluxes(r_bands)
# remove_unneeded_fluxes(i_bands)
# remove_unneeded_fluxes(z_bands)
# remove_unneeded_fluxes(y_bands)

In [12]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")

In [13]:
cigale_input = Table()

for tile in file_list:
    
    file_num = tile.split('_')[4].split('.')[0]
    print("Tile {}:".format(tile.split('/')[-1]))
    best_catalogue = Table.read(tile)
    best_catalogue = main_help_sample(best_catalogue)


    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

    print("Number of sources with redshift in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)
    remove_unneeded_fluxes(j_bands)
    remove_unneeded_fluxes(h_bands)
    remove_unneeded_fluxes(k_bands)

    cigale_input = vstack([cigale_input, best_catalogue])
    #best_catalogue.write("data_tmp/tiles/XMM-LSS_cigale_best_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    
    
cigale_input.write("data_tmp/xmm-lss_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

Tile xmm-lss_20190207_cigale_60.fits:
Number of good sources: 1195
Number of sources with redshift in file 60: 1119


INFO:herschelhelp.external:For 9 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 6 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, 

Tile xmm-lss_20190207_cigale_9.fits:
Number of good sources: 220
Number of sources with redshift in file 9: 217


INFO:herschelhelp.external:For 2 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, 

Tile xmm-lss_20190207_cigale_51.fits:
Number of good sources: 1461
Number of sources with redshift in file 51: 1304


...

INFO:herschelhelp.external:For 3 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, 

Tile xmm-lss_20190207_cigale_14.fits:
Number of good sources: 505
Number of sources with redshift in file 14: 468


INFO:herschelhelp.external:For 5 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, 

Tile xmm-lss_20190207_cigale_7.fits:
Number of good sources: 13
Number of sources with redshift in file 7: 13
Tile xmm-lss_20190207_cigale_87.fits:
Number of good sources: 0
Number of sources with redshift in file 87: 0
Tile xmm-lss_20190207_cigale_25.fits:
Number of good sources: 1055
Number of sources with redshift in file 25: 959


INFO:herschelhelp.external:For 12 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 9 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources,

# All optnir objects with photo-z

In [14]:
cigale_input = Table()

for tile in file_list:
    
    file_num = tile.split('_')[4].split('.')[0]
    print("Tile {}:".format(tile.split('/')[-1]))
    best_catalogue = Table.read(tile)
    
    #best_catalogue = main_help_sample(best_catalogue)
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    best_catalogue = best_catalogue[optnir]

    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

    print("Number of sources with redshift in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)
    remove_unneeded_fluxes(j_bands)
    remove_unneeded_fluxes(h_bands)
    remove_unneeded_fluxes(k_bands)

    cigale_input = vstack([cigale_input, best_catalogue])
    #best_catalogue.write("data_tmp/tiles/XMM-LSS_cigale_best_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    
    
cigale_input.write("data_tmp/xmm-lss_cigale_optnir_extcor_{}.fits".format(SUFFIX), overwrite=True)

Tile xmm-lss_20190207_cigale_60.fits:
Number of sources with redshift in file 60: 22796


INFO:herschelhelp.external:For 566 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 437 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 344 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 346 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 341 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3

Tile xmm-lss_20190207_cigale_9.fits:
Number of sources with redshift in file 9: 3690


INFO:herschelhelp.external:For 21 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 20 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 19 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 19 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 19 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 19 sou

Tile xmm-lss_20190207_cigale_51.fits:
Number of sources with redshift in file 51: 20534


...

INFO:herschelhelp.external:For 195 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 177 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 165 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 165 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 165 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1

Tile xmm-lss_20190207_cigale_7.fits:
Number of sources with redshift in file 7: 2890


INFO:herschelhelp.external:For 3 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, 

Tile xmm-lss_20190207_cigale_87.fits:
Number of sources with redshift in file 87: 217
Tile xmm-lss_20190207_cigale_25.fits:
Number of sources with redshift in file 25: 23286


INFO:herschelhelp.external:For 183 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 161 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 148 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 148 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 148 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1

# Catalogue using spectroscopic redshift

In [15]:
cigale_input = Table()

for tile in file_list:
    
    file_num = tile.split('_')[4].split('.')[0]
    print("Tile {}:".format(tile.split('/')[-1]))
    best_catalogue = Table.read(tile)
    best_catalogue = main_help_sample(best_catalogue)
    
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    best_catalogue = best_catalogue[optnir]

    best_catalogue.remove_column("redshift")
    best_catalogue["zspec"].name = "redshift"
    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

    print("Number of sources with redshift in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)
    remove_unneeded_fluxes(j_bands)
    remove_unneeded_fluxes(h_bands)
    remove_unneeded_fluxes(k_bands)

    cigale_input = vstack([cigale_input, best_catalogue])
    #best_catalogue.write("data_tmp/tiles/XMM-LSS_cigale_best_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    
    
cigale_input.write("data_tmp/xmm-lss_cigale_optnir_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)

Tile xmm-lss_20190207_cigale_60.fits:
Number of good sources: 1195
Number of sources with redshift in file 60: 140
Tile xmm-lss_20190207_cigale_9.fits:
Number of good sources: 220
Number of sources with redshift in file 9: 5
Tile xmm-lss_20190207_cigale_51.fits:
Number of good sources: 1461
Number of sources with redshift in file 51: 252
Tile xmm-lss_20190207_cigale_78.fits:
Number of good sources: 324
Number of sources with redshift in file 78: 35
Tile xmm-lss_20190207_cigale_33.fits:
Number of good sources: 635
Number of sources with redshift in file 33: 95
Tile xmm-lss_20190207_cigale_49.fits:
Number of good sources: 1149
Number of sources with redshift in file 49: 189


INFO:herschelhelp.external:For 1 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Tile xmm-lss_20190207_cigale_55.fits:
Number of good sources: 1494
Number of sources with redshift in file 55: 278


INFO:herschelhelp.external:For 2 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, 

Tile xmm-lss_20190207_cigale_64.fits:
Number of good sources: 1215
Number of sources with redshift in file 64: 82


...

INFO:herschelhelp.external:For 2 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Tile xmm-lss_20190207_cigale_76.fits:
Number of good sources: 589
Number of sources with redshift in file 76: 75
Tile xmm-lss_20190207_cigale_14.fits:
Number of good sources: 505
Number of sources with redshift in file 14: 80


INFO:herschelhelp.external:For 3 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, 

Tile xmm-lss_20190207_cigale_7.fits:
Number of good sources: 13
Number of sources with redshift in file 7: 0
Tile xmm-lss_20190207_cigale_87.fits:
Number of good sources: 0
Number of sources with redshift in file 87: 0
Tile xmm-lss_20190207_cigale_25.fits:
Number of good sources: 1055
Number of sources with redshift in file 25: 252


INFO:herschelhelp.external:For 4 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band megacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, 